# Object Detection Demo
Welcome to the object detection task walkthrough!  This notebook will walk you step by step through the process of training a fast-rcnn model for object detection, evaluating it during the training, and using a pre-trained model to detect objects in some images. The object detection demo uses the Tensorflow Object Detection API, which is an open source framework built on top of TensorFlow that makes it easy to construct, train and deploy object detection models. 

# Preparing Dataset
Training requires datasets, and object detection model training requires a lot of images with labeled objects. There are some famous datasets used for computer vision: 

Dataset | Training Set Size | Testing Set Size | Number of Classes | Comments
:------:|:---------------:|:---------------------:|:-----------:|:-----------:
Flowers|2500 | 2500 | 5 | Various sizes (source: Flickr)
[Cifar10](https://www.cs.toronto.edu/~kriz/cifar.html) | 60k| 10k | 10 |32x32 color
[MNIST](http://yann.lecun.com/exdb/mnist/)| 60k | 10k | 10 | 28x28 gray
[ImageNet](http://www.image-net.org/challenges/LSVRC/2012/)|1.2M| 50k | 1000 | Various sizes

Datasets used for object detection are generally composed of raw images, bounding boxes and classifications. Here, we are using the Oxford-IIIT Pets dataset. Here's a sample from the dataset:
<img src="http://i64.tinypic.com/30rpbaw.jpg">

The raw dataset for Oxford-IIIT Pets lives
[here](http://www.robots.ox.ac.uk/~vgg/data/pets/). Both the image dataset [`images.tar.gz`](http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz)
and the groundtruth data [`annotations.tar.gz`](http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz) are needed. They have been downloaded and unzipped in the path `./data`. 

For now, the file structure should appear as follows:
```lang-none
+ ${YOUR_PATH}/
    + data/
      + images/
      + annotations/
    + models/  
    + config/  
    + code/ 
    + test_images/
    - object_detection_demo.ipynb
    - evaluate.ipynb
    - model_inference.ipynb
```

In [1]:
!ls ./data
#you can use more bash commands here to explore the file structure of the dataset.

annotations  annotations.tar.gz  images  images.tar.gz


The Tensorflow Object Detection API expects data to be in the TFRecord format, a general format used for Tensorflow. Except for the raw data, we need a label map mapping class name to ids. It is constructed according to the format of Google Protocol Buffer(protobuf) as follows:

In [2]:
!cp code/models/research/object_detection/data/pet_label_map.pbtxt ./data/pet_label_map.pbtxt
!cat ./data/pet_label_map.pbtxt

item {
  id: 1
  name: 'Abyssinian'
}

item {
  id: 2
  name: 'american_bulldog'
}

item {
  id: 3
  name: 'american_pit_bull_terrier'
}

item {
  id: 4
  name: 'basset_hound'
}

item {
  id: 5
  name: 'beagle'
}

item {
  id: 6
  name: 'Bengal'
}

item {
  id: 7
  name: 'Birman'
}

item {
  id: 8
  name: 'Bombay'
}

item {
  id: 9
  name: 'boxer'
}

item {
  id: 10
  name: 'British_Shorthair'
}

item {
  id: 11
  name: 'chihuahua'
}

item {
  id: 12
  name: 'Egyptian_Mau'
}

item {
  id: 13
  name: 'english_cocker_spaniel'
}

item {
  id: 14
  name: 'english_setter'
}

item {
  id: 15
  name: 'german_shorthaired'
}

item {
  id: 16
  name: 'great_pyrenees'
}

item {
  id: 17
  name: 'havanese'
}

item {
  id: 18
  name: 'japanese_chin'
}

item {
  id: 19
  name: 'keeshond'
}

item {
  id: 20
  name: 'leonberger'
}

item {
  id: 21
  name: 'Maine_Coon'
}

item {
  id: 22
  name: 'miniature_pinscher'
}

item {
  id: 23
  name: 'newfoundland'
}

item {
  id: 24
  name: 'Persian'
}

item 

We can now run the `create_pet_tf_record` script to convert from the raw Oxford-IIIT Pet dataset into TFRecords, you have to fill in the data directory and the output direcotry for your TFRecord files, we recommend you to put the TFRecord files in the `./data` path, but you can try another path: 

In [7]:
!python code/models/research/object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path=./data/pet_label_map.pbtxt \
    --data_dir=./data \
    --output_dir=./data 
# Note: It is normal to see some warnings when running this script. You may ignore them.

/qydata/htianab/code/models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


The dataset is partitioned into two TFRecord files named `pet_train.record` with 70% images and `pet_val.record` with 30% images, which should be generated in the output directory you set. `pet_train.record` is used for training your model and `pet_val.record` is for evaluating the performance of your model. 

In [9]:
!ls -al ./data

total 1161652
drwx------ 4 htianab cs      4096 May  2 15:15 .
drwx------ 8 htianab cs      4096 May  2 15:16 ..
drwx------ 4 htianab cs       141 May  2 14:54 annotations
-rw------- 1 htianab cs  19173078 Jun 30  2012 annotations.tar.gz
drwx------ 2 htianab cs    253952 Jun 19  2012 images
-rw------- 1 htianab cs 791918971 Jun 19  2012 images.tar.gz
-rw------- 1 htianab cs      1532 May  2 15:05 pet_label_map.pbtxt
-rw------- 1 htianab cs 264907960 May  2 15:15 pet_train.record
-rw------- 1 htianab cs 113175175 May  2 15:15 pet_val.record


# Training & Evaluation

## Overview

The Tensorflow Object Detection API uses protobuf files to configure the
training and evaluation process. The schema for the training pipeline can be
found in `/code/models/research/object_detection/protos/pipeline.proto`. At a high level, the config
file is split into 5 parts:

1. The `model` configuration. This defines what type of model will be trained (ie. meta-architecture, feature extractor).
2. The `train_config`, which decides what parameters should be used to train model parameters (ie. the training speed, input preprocessing).
3. The `eval_config`, which determines what set of metrics will be reported for evaluation.
4. The `train_input_config`, which defines what dataset the model should be trained on.
5. The `eval_input_config`, which defines what dataset the model will be evaluated on. Typically this should be different than the training input dataset.

Sample model configurations have been provided
in the `code/models/research/object_detection/samples/configs` folder. Different training models need different configurations. Here we use the faster RCNN model with inception v2 neural network for training, whose configuration file is `faster_rcnn_inception_v2_pets.config`. You can copy it into the `./config` directory and check the parameters in the file.

In [10]:
!cp code/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config ./config

## Picking Model Parameters

The model parameters in `model` part in sample configuration has been set properly for training. The only parameter you should change is the `num_classes` field should be changed to a value suited for the dataset the user is training on. You can check the label map file for the number of classes.

## Defining Inputs

The Tensorflow Object Detection API accepts inputs in the TFRecord file format.
Users must specify the locations of both the training and evaluation files.
Additionally, users should also specify a label map, which define the mapping
between a class id and class name. The label map should be identical between
training and evaluation datasets.

An example input configuration looks as follows:

```
tf_record_input_reader {
  input_path: "/usr/home/username/data/train.record"
}
label_map_path: "/usr/home/username/data/label_map.pbtxt"
```
Change the `tf_record_input_reader` parameters in both `train_input_config` and `eval_input_config` parts. Note that the label map file `label_map_path` is generally the same for the training and evaluation processes.

## Configuring the Trainer

The `train_config` defines parts of the training process:

1. Model parameter initialization.
2. Input preprocessing.
3. SGD parameters.

### Model Parameter Initialization

While optional, it is highly recommended that users utilize other object
detection checkpoints. Training an object detector from scratch can take days.
To speed up the training process, it is recommended that users re-use the
feature extractor parameters from a pre-existing object classification or
detection checkpoint. `fine_tune_checkpoint` provide a path to
the pre-existing checkpoint. Here we have downloaded a faster RCNN model with inception v2 neural network pre-trained with COCO dataset. You can find it in `./models/`

Change the `fine_tune_checkpoint` parameter to the downloaded checkpoint file.

### Input Preprocessing

The `data_augmentation_options` in `train_config` can be used to specify
how training data can be transformed to augment the dataset.

### SGD Parameters

The remaining parameters in `train_config` are hyperparameters for gradient
descent, representing the training speed. Here we change the `num_steps` to xxx to limit the training steps. If not set, the training job will
run indefinitely until the user kills it. 

## Configuring the Evaluator

Currently evaluation is fixed to generating metrics as defined by the PASCAL VOC
challenge. The parameters for `eval_config` are set to reasonable defaults and
typically do not need to be configured.

## Train!

After filling the configuration file, let's create two directories for saving the checkpoints of training model and the results of evaluation. 

In [11]:
!mkdir ./models/faster_rcnn_inception_v2_coco_2018_01_28/train
!mkdir ./models/faster_rcnn_inception_v2_coco_2018_01_28/eval

mkdir: cannot create directory ‘./models/faster_rcnn_inception_v2_coco_2018_01_28/train’: File exists
mkdir: cannot create directory ‘./models/faster_rcnn_inception_v2_coco_2018_01_28/eval’: File exists


the file structure should appear as follows:
```
+data
  - label_map file
  - train TFRecord file
  - eval TFRecord file
+models
  + faster_rcnn_inception_v2_coco_2018_01_28
    +train
    +eval
+config
  - pipeline config file
```
and now we can start the training!

In [17]:
# Our GPUs on server are not sufficient for all students in the workshop, so we set the environment variable to running tensorflow on CPU.
!export CUDA_VISIBLE_DEVICES="-1"

In [ ]:
!python ./code/models/research/object_detection/train.py \
    --logtostderr \
    --pipeline_config_path=./config/faster_rcnn_inception_v2_pets.config \
    --train_dir=./models/faster_rcnn_inception_v2_coco_2018_01_28/train

Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

/qydata/htianab/software/anaconda3/envs/py3tfCPU/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Instructions for updating:
Please switch to tf.train.get_or_create_global_step


Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2018-05-02 18:31:17.499230: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
INFO:tensorflow:Restoring parameters from ./models/faster_rcnn_inception_v2_coco_2018_01_28/train/model.ckpt-0
INFO:tensorflow:Restoring parameters from ./models/faster_rcnn_inception_v2_coco_2018_01_28/train/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path ./models/faster_rcnn_inception_v2_coco_2018_01_28/train/model.ckpt
INFO:tensorflow:Saving checkpoint to path ./models/faster_rcnn_inception_v2_coco_2018_01_28/train/model.c

INFO:tensorflow:global step 53: loss = 3.1068 (1.272 sec/step)
INFO:tensorflow:global step 53: loss = 3.1068 (1.272 sec/step)
INFO:tensorflow:global step 54: loss = 0.6959 (1.339 sec/step)
INFO:tensorflow:global step 54: loss = 0.6959 (1.339 sec/step)
INFO:tensorflow:global step 55: loss = 1.5993 (1.262 sec/step)
INFO:tensorflow:global step 55: loss = 1.5993 (1.262 sec/step)
INFO:tensorflow:global step 56: loss = 2.3901 (1.474 sec/step)
INFO:tensorflow:global step 56: loss = 2.3901 (1.474 sec/step)
INFO:tensorflow:global step 57: loss = 2.2721 (1.301 sec/step)
INFO:tensorflow:global step 57: loss = 2.2721 (1.301 sec/step)
INFO:tensorflow:global step 58: loss = 1.9570 (1.370 sec/step)
INFO:tensorflow:global step 58: loss = 1.9570 (1.370 sec/step)
INFO:tensorflow:global step 59: loss = 1.4691 (1.256 sec/step)
INFO:tensorflow:global step 59: loss = 1.4691 (1.256 sec/step)
INFO:tensorflow:global step 60: loss = 1.2054 (1.336 sec/step)
INFO:tensorflow:global step 60: loss = 1.2054 (1.336 se

INFO:tensorflow:global step 117: loss = 2.7341 (1.334 sec/step)
INFO:tensorflow:global step 117: loss = 2.7341 (1.334 sec/step)
INFO:tensorflow:global step 118: loss = 1.4742 (1.407 sec/step)
INFO:tensorflow:global step 118: loss = 1.4742 (1.407 sec/step)
INFO:tensorflow:global step 119: loss = 1.9249 (1.309 sec/step)
INFO:tensorflow:global step 119: loss = 1.9249 (1.309 sec/step)
INFO:tensorflow:global step 120: loss = 1.7005 (1.261 sec/step)
INFO:tensorflow:global step 120: loss = 1.7005 (1.261 sec/step)
INFO:tensorflow:global step 121: loss = 1.2136 (1.295 sec/step)
INFO:tensorflow:global step 121: loss = 1.2136 (1.295 sec/step)
INFO:tensorflow:global step 122: loss = 0.9088 (1.273 sec/step)
INFO:tensorflow:global step 122: loss = 0.9088 (1.273 sec/step)
INFO:tensorflow:global step 123: loss = 1.4421 (1.354 sec/step)
INFO:tensorflow:global step 123: loss = 1.4421 (1.354 sec/step)
INFO:tensorflow:global step 124: loss = 2.6296 (1.344 sec/step)
INFO:tensorflow:global step 124: loss = 

INFO:tensorflow:global step 180: loss = 1.1677 (1.292 sec/step)
INFO:tensorflow:global step 180: loss = 1.1677 (1.292 sec/step)
INFO:tensorflow:global step 181: loss = 1.7205 (1.359 sec/step)
INFO:tensorflow:global step 181: loss = 1.7205 (1.359 sec/step)
INFO:tensorflow:global step 182: loss = 1.2405 (1.391 sec/step)
INFO:tensorflow:global step 182: loss = 1.2405 (1.391 sec/step)
INFO:tensorflow:global step 183: loss = 1.2461 (1.375 sec/step)
INFO:tensorflow:global step 183: loss = 1.2461 (1.375 sec/step)
INFO:tensorflow:global step 184: loss = 2.2116 (1.414 sec/step)
INFO:tensorflow:global step 184: loss = 2.2116 (1.414 sec/step)
INFO:tensorflow:global step 185: loss = 0.4758 (1.411 sec/step)
INFO:tensorflow:global step 185: loss = 0.4758 (1.411 sec/step)
INFO:tensorflow:global step 186: loss = 2.1636 (1.486 sec/step)
INFO:tensorflow:global step 186: loss = 2.1636 (1.486 sec/step)
INFO:tensorflow:global step 187: loss = 1.8985 (1.289 sec/step)
INFO:tensorflow:global step 187: loss = 

INFO:tensorflow:global step 245: loss = 1.9036 (1.335 sec/step)
INFO:tensorflow:global step 245: loss = 1.9036 (1.335 sec/step)
INFO:tensorflow:global step 246: loss = 1.0246 (1.335 sec/step)
INFO:tensorflow:global step 246: loss = 1.0246 (1.335 sec/step)
INFO:tensorflow:global step 247: loss = 1.0883 (1.328 sec/step)
INFO:tensorflow:global step 247: loss = 1.0883 (1.328 sec/step)
INFO:tensorflow:global step 248: loss = 1.8841 (1.355 sec/step)
INFO:tensorflow:global step 248: loss = 1.8841 (1.355 sec/step)
INFO:tensorflow:global step 249: loss = 0.3335 (1.367 sec/step)
INFO:tensorflow:global step 249: loss = 0.3335 (1.367 sec/step)
INFO:tensorflow:global step 250: loss = 2.0010 (1.403 sec/step)
INFO:tensorflow:global step 250: loss = 2.0010 (1.403 sec/step)
INFO:tensorflow:global step 251: loss = 1.4469 (1.273 sec/step)
INFO:tensorflow:global step 251: loss = 1.4469 (1.273 sec/step)
INFO:tensorflow:global step 252: loss = 0.3736 (1.448 sec/step)
INFO:tensorflow:global step 252: loss = 

INFO:tensorflow:global step 308: loss = 0.2833 (2.194 sec/step)
INFO:tensorflow:global step 308: loss = 0.2833 (2.194 sec/step)
INFO:tensorflow:global step 309: loss = 0.5351 (2.111 sec/step)
INFO:tensorflow:global step 309: loss = 0.5351 (2.111 sec/step)
INFO:tensorflow:global step 310: loss = 0.6494 (2.084 sec/step)
INFO:tensorflow:global step 310: loss = 0.6494 (2.084 sec/step)
INFO:tensorflow:global step 311: loss = 0.5042 (2.065 sec/step)
INFO:tensorflow:global step 311: loss = 0.5042 (2.065 sec/step)
INFO:tensorflow:global step 312: loss = 1.0382 (2.105 sec/step)
INFO:tensorflow:global step 312: loss = 1.0382 (2.105 sec/step)
INFO:tensorflow:global step 313: loss = 1.6563 (2.048 sec/step)
INFO:tensorflow:global step 313: loss = 1.6563 (2.048 sec/step)
INFO:tensorflow:global step 314: loss = 1.9440 (2.110 sec/step)
INFO:tensorflow:global step 314: loss = 1.9440 (2.110 sec/step)
INFO:tensorflow:global step 315: loss = 0.2135 (2.144 sec/step)
INFO:tensorflow:global step 315: loss = 

INFO:tensorflow:global step 371: loss = 0.1755 (1.589 sec/step)
INFO:tensorflow:global step 371: loss = 0.1755 (1.589 sec/step)
INFO:tensorflow:global step 372: loss = 0.3712 (1.283 sec/step)
INFO:tensorflow:global step 372: loss = 0.3712 (1.283 sec/step)
INFO:tensorflow:global step 373: loss = 0.5762 (1.319 sec/step)
INFO:tensorflow:global step 373: loss = 0.5762 (1.319 sec/step)
INFO:tensorflow:global step 374: loss = 0.4417 (1.367 sec/step)
INFO:tensorflow:global step 374: loss = 0.4417 (1.367 sec/step)
INFO:tensorflow:global step 375: loss = 0.6485 (1.373 sec/step)
INFO:tensorflow:global step 375: loss = 0.6485 (1.373 sec/step)
INFO:tensorflow:global step 376: loss = 1.4901 (1.337 sec/step)
INFO:tensorflow:global step 376: loss = 1.4901 (1.337 sec/step)
INFO:tensorflow:global step 377: loss = 2.0483 (1.380 sec/step)
INFO:tensorflow:global step 377: loss = 2.0483 (1.380 sec/step)
INFO:tensorflow:global step 378: loss = 0.1602 (1.528 sec/step)
INFO:tensorflow:global step 378: loss = 

INFO:tensorflow:global step 432: loss = 0.9205 (1.501 sec/step)
INFO:tensorflow:global step 432: loss = 0.9205 (1.501 sec/step)
INFO:tensorflow:global step 433: loss = 0.8452 (1.407 sec/step)
INFO:tensorflow:global step 433: loss = 0.8452 (1.407 sec/step)
INFO:tensorflow:global step 434: loss = 0.1622 (1.460 sec/step)
INFO:tensorflow:global step 434: loss = 0.1622 (1.460 sec/step)
INFO:tensorflow:global step 435: loss = 0.3213 (1.349 sec/step)
INFO:tensorflow:global step 435: loss = 0.3213 (1.349 sec/step)
INFO:tensorflow:global step 436: loss = 0.5182 (1.379 sec/step)
INFO:tensorflow:global step 436: loss = 0.5182 (1.379 sec/step)
INFO:tensorflow:global step 437: loss = 0.4270 (1.400 sec/step)
INFO:tensorflow:global step 437: loss = 0.4270 (1.400 sec/step)
INFO:tensorflow:global step 438: loss = 0.5139 (1.456 sec/step)
INFO:tensorflow:global step 438: loss = 0.5139 (1.456 sec/step)
INFO:tensorflow:global step 439: loss = 1.1056 (1.388 sec/step)
INFO:tensorflow:global step 439: loss = 

where `${PATH_TO_YOUR_PIPELINE_CONFIG}` points to the pipeline config and
`${PATH_TO_TRAIN_DIR}` points to the directory in which training checkpoints
and events will be written to. 

## Evaluate!

Evaluation is run as a separate job. The eval job will periodically poll the
train directory for new checkpoints and evaluate them on a test dataset. So it should run synchronously with the training job. Execute the following command:

```bash
# From the tensorflow/models/research/ directory
python ./code/models/research/object_detection/eval.py \
    --logtostderr \
    --pipeline_config_path=${PATH_TO_YOUR_PIPELINE_CONFIG} \
    --checkpoint_dir=${PATH_TO_TRAIN_DIR} \
    --eval_dir=${PATH_TO_EVAL_DIR}
```

where `${PATH_TO_YOUR_PIPELINE_CONFIG}` points to the pipeline config,
`${PATH_TO_TRAIN_DIR}` points to the directory in which training checkpoints
were saved (same as the training job) and `${PATH_TO_EVAL_DIR}` points to the
directory in which evaluation events will be saved. Because one notebook has only one thread, you can run the command in shell, or in another notebook [here](evaluate.ipynb).

Progress for training and eval jobs can be inspected using Tensorboard. If
using the recommended directory structure, Tensorboard can be run using the
following command:

```bash
tensorboard --logdir=${PATH_TO_MODEL_DIRECTORY}
```

## Visualize the Results with TensorBoard

Progress for training and eval jobs can be inspected using Tensorboard. If
using the recommended directory structure, Tensorboard can be run using the
following command:

```bash
tensorboard --logdir=${PATH_TO_MODEL_DIRECTORY}
```

where `${PATH_TO_MODEL_DIRECTORY}` points to the directory that contains the
train and eval directories. Please note it may take Tensorboard a couple minutes
to populate with data.

# Exporting a trained model for inference

After your model has been trained, you should export it to a Tensorflow
graph proto. A checkpoint will typically consist of three files:
```
* model.ckpt-${CHECKPOINT_NUMBER}.data-00000-of-00001,
* model.ckpt-${CHECKPOINT_NUMBER}.index
* model.ckpt-${CHECKPOINT_NUMBER}.meta
```
After you've identified a candidate checkpoint to export, run the following
command:

In [23]:
!python ./code/models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path ${PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_prefix ${TRAIN_PATH} \
    --output_directory ./models/faster_rcnn_inception_v2_coco_2018_01_28/output_inference_graph.pb

Traceback (most recent call last):
  File "/qydata/htianab/software/anaconda3/envs/py3tf/lib/python3.6/site-packages/absl/flags/_flagvalues.py", line 525, in _assert_validators
    validator.verify(self)
  File "/qydata/htianab/software/anaconda3/envs/py3tf/lib/python3.6/site-packages/absl/flags/_validators.py", line 81, in verify
    raise _exceptions.ValidationError(self.message)
absl.flags._exceptions.ValidationError: Flag --trained_checkpoint_prefix must be specified.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "./code/models/research/object_detection/export_inference_graph.py", line 147, in <module>
    tf.app.run()
  File "/qydata/htianab/software/anaconda3/envs/py3tf/lib/python3.6/site-packages/tensorflow/python/platform/app.py", line 118, in run
    argv = flags.FLAGS(_sys.argv if argv is None else argv, known_only=True)
  File "/qydata/htianab/software/anaconda3/envs/py3tf/lib/python3.6/site-packages/tensorflo

Afterwards, you should see a graph named output_inference_graph.pb.